```{currentmodule} optimap
```

In [ ]:
# Code snippet for rendering animations in the docs
from IPython.display import HTML
import warnings
import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 2**128
import matplotlib.pyplot as plt

def render_ani_func(f):
    om.utils.disable_interactive_backend_switching()
    plt.switch_backend('Agg')
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        ani = f()
    %matplotlib inline
    om.utils.enable_interactive_backend_switching()

    vid = HTML(ani.to_html5_video())
    plt.close('all')
    return vid

```{tip}
Download this tutorial as a {download}`Jupyter notebook <converted/ratiometry.ipynb>`, or a {download}`python script <converted/ratiometry.py>` with code cells.
```

# Tutorial 7: Ratiometry

## Combining Numerical Motion Compensation and Ratiometry with Di-4-ANEPPS

In this tutorial, we will discuss the analysis of ratiometric optical mapping data and the benefits of a combined approach consisting of numerical motion tracking and ratiometric imaging. Numerical motion tracking and motion-stabilization alone cannot inhibit motion artifacts entirely, see Tutorials [1](basics.ipynb) & [4](motion_compensation.ipynb). Numerical motion tracking alone cannot remove all motion artifacts alone, because it does not remove the relative motion between the tissue and the light sources used to excite the fluorescent dye. Motion tracking is merely a change of the frame of reference. The physical relative motion persists afer numerical motion-stabilization and causes fluctuations in the illumination. Typically, what can be seen in motion-stabilized videos is that the illumination (e.g. a Gaussian intensity distribution) moves back and forth across the tissue which leads to illumination artifacts. Motion artifacts therefore often have two components: dissociation-related motion artifacts and illumination artifacts. To achieve better artifact compensation, particularly with stronger motion, it is therefore required to combine numerical motion tracking with ratiometric imaging, as described in {cite:t}`Kappadan2020` or in {cite:t}`Zhang2016`. 

In the example below, the heart was stained with Di-4-ANEPPS and imaged using alternating green and blue illumination at 500fps. More specifically, the tissue was illuminated in every odd frame with green light and in every even frame with blue light, respectively. This approach is also referred to as 'excitation ratiometry'. After numerical motion tracking and stabilization, the green and blue videos are divided by each other to obtain a motion-stabilized ratiometric video. Using this method, both motion artifacts as well as illumination artifacts (which also lead to motion artifacts) can be significantly reduced. 

First, we load the video data, which is stored as a single file (`Example_05_Ratiometry.npy`) containing the two green and blue videos in an interleaved fashion (frame 1 = green, frame 2 = blue, frame 3 = green, etc.). Using optimap's {func}`load_video()` function, we can specify to load only every 2nd frame and to start reading from the first or the second frame, respectively:

In [ ]:
import optimap as om

filename = om.utils.retrieve_example_data('Example_05_Ratiometry.npy')
video_blue = om.load_video(filename, start_frame=0, step=2)
video_green = om.load_video(filename, start_frame=1, step=2)

om.print_properties(video_blue)
om.print_properties(video_green)


Alternatively, one could load the video and manually split the video into the green and blue videos (`video_green = video[::2,:,:]` etc.). Both videos contain 'uint16' intensity values in each pixel, which means that they can have whole unsigned integer values between 0 and 65535. The videos dimensions are 128 x 128 pixels and each video contains 600 frames. The original interleaved video contains 1200 frames. The green video is only slightly brighter based on the maximum value.

When we play the two videos next to each other we notice that there is signal in only the green video but not in the blue video:

In [ ]:
om.video.play2(video_green, video_blue,
               title1="green video with signal",
               title2="blue video without signal");

In [ ]:
render_ani_func(lambda: om.video.play2(video_green, video_blue, title1="green video with signal", title2="blue video without signal", interval=20))

If we plot and compare two optical traces from the green and blue videos respectively, we can see not in all but in most areas stronger intensity fluctuations in the green video:

In [ ]:
om.trace.set_default_trace_window('disc') # the trace is sample from a disc-shaped region with a diameter specified by 'size'
om.compare_traces([video_green, video_blue],
                  labels=['green','blue'],
                  colors=['green','blue'])

Keep in mind that the motion in both videos is not compensated yet and produces strong motion artifacts. In the green video, the intensity fluctuations are caused in parts by motion and in parts by the optical signal related to the action potential. In the blue video, nearly all fluctuations are caused by motion. Let's look at a few specific examples. Above, the function {func}`compare_traces()` was started in interactive mode because we did not provide specific coordinates as arguments. We can also provide specific coordinates, e.g. the pixel at (50,50) or a list of pixels, as follows:

In [ ]:
positions = [(77, 57), (72, 74), (65, 42), (80, 75), (90, 90)]
om.compare_traces([video_green, video_blue],
                  positions,
                  labels=['green','blue'],
                  colors=['green', 'blue'],
                  fps=250)

In the some of these examples, the green optical trace shows steep downstrokes which is typical for the upstroke of the action potential, whereas the blue trace does not exhibit such downstrokes and exhibits much weaker and inconsistent intensity fluctuations. These fluctuations are largely caused by motion, whereas the green trace has a signal component that is caused by the fluorescent dye. You can best explore this behavior using the interactive mode. We can visualize from where these traces were sampled as follows:

In [ ]:
om.show_positions(video_green[0], positions)

Now let's perform numerical motion tracking and motion-stabilization with the green and blue videos:

In [ ]:
video_warped_green = om.motion_compensate(video_green, contrast_kernel=5, ref_frame=0)
video_warped_blue = om.motion_compensate(video_blue, contrast_kernel=5, ref_frame=0)
om.video.playn([video_green, video_warped_green, video_blue, video_warped_blue],
               titles=["video green", "stabilized video green", "video blue", "stabilized video blue"],
               figsize=(8, 3));

In [ ]:
video_warped_green = om.motion_compensate(video_green, contrast_kernel=5, ref_frame=0)
video_warped_blue = om.motion_compensate(video_blue, contrast_kernel=5, ref_frame=0)
render_ani_func(lambda: om.video.playn([video_green, video_warped_green, video_blue, video_warped_blue],
               titles=["video green", "stabilized video green", "video blue", "stabilized video blue"],
               figsize=(8, 3)))

In the comparison above, you can see that both the green and blue videos were succesfully tracked and warped (motion-stabilized). There is no residual motion after warping and there are no tracking artifacts. Please refer to {cite:t}`Christoph2018a,Kappadan2020,Lebert2022` for details. Internally, the function {func}`motion_compensate()` computed a contrast-enhanced video, registered / tracked and stabilized the motion / warped the original video. Now let's look at how the green and blue traces have changed after the motion-stabilization:

In [ ]:
om.compare_traces([video_green, video_warped_green],
                  positions,
                  labels=['green','warped green'],
                  size=3,
                  colors=['lightgreen', 'green'])

We plotted the motion-stabilized green traces slightly darker than the original green trace. In this example, the motion-stabilized traces are more consistent, exhibit less variability and less deflections in the repolarization phase of the action potential. In one example, the action potential shape was not visible at all before motion-stabilization, but becomes visible after motion-stabilization. The blue traces change as follows:

In [ ]:
om.compare_traces([video_blue, video_warped_blue],
                  positions,
                  labels=['blue','warped blue'],
                  size=3,
                  colors=['lightblue', 'blue'])

We plotted the motion-stabilized blue traces in dark blue and the original traces in light/grey blue. Here the changes are not obvious or systematic, because we are largely looking at motion artifacts. If anything then the traces become more regular. You can explore these changes yourself using the interactive mode of the {func}`compare_traces()` function:

In [ ]:
om.compare_traces([video_green, video_warped_green],
                  labels=['green','warped green'],
                  size=3,
                  colors=['lightgreen', 'green'],
                  fps=250)


Now we can combine the green and blue motion-stabilized videos to obtain a motion-stabilized ratiometric video:

In [ ]:
video_warped_ratio = video_warped_green/video_warped_blue
om.video.playn([video_green, video_warped_green, video_warped_blue, video_warped_ratio],
               titles=["video green", "stabilized video green", "stabilized video blue", "stabilized video ratio"],
               figsize=(8, 3));

In [ ]:
video_warped_ratio = video_warped_green/video_warped_blue
render_ani_func(lambda: om.video.playn([video_green, video_warped_green, video_warped_blue, video_warped_ratio],
               titles=["video green", "stabilized video green", "stabilized video blue", "stabilized video ratio"], 
               figsize=(8, 3),
               interval=20))

To be able to compare the motion-stabilized green traces with the corresponding motion-stabilized ratiometric traces, we need to renormalize the videos:


In [ ]:
video_warped_green_norm = om.video.normalize_pixelwise_slidingwindow(video_warped_green, window_size=100)
video_warped_ratio_norm = om.video.normalize_pixelwise_slidingwindow(video_warped_ratio, window_size=100)

#norm_raw = om.video.normalize_pixelwise_slidingwindow(video, window_size=60)

Now if we compare the motion-stabilized green traces with the corresponding motion-stabilized ratiometric traces we can see slight improvements of the signal quality:

In [ ]:
om.compare_traces([video_warped_green_norm, video_warped_ratio_norm],
                  positions,
                  labels=['warped green','ratio'],
                  size=3,
                  colors=['g', 'k'])

These subtle improvements become more evident when you explore the data yourself:

In [ ]:
om.compare_traces([video_warped_green_norm, video_warped_ratio_norm],
                  labels=['green','ratio'],
                  size=3,
                  colors=['green', 'k'],
                  fps=250)

and zoom in:

In [ ]:
t = 150
om.compare_traces([video_warped_green_norm[:t], video_warped_ratio_norm[:t]],
                  labels=['green','ratio'],
                  colors=['green', 'k'],
                  size=3,
                  fps=250)

The action potential duration (APD) is more heterogeneous in non-ratiometric motion-stabilized videos.

In [ ]:
traces_blue, positions = om.select_traces(video_blue)
traces_green = om.extract_traces(video_green, positions)

TO DO: use flow field from blue channle to compensate the green channel